## Importing Libraries

In [ ]:
!pip install opencv-python

In [1]:
import cv2
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt

## Datasets and Labels

In [7]:
image_directory = 'Datasets/'

In [8]:
no_tumour_images = os.listdir(image_directory + 'no/')
yes_tumour_images = os.listdir(image_directory + 'yes/')

In [9]:
dataset = []
label = []
INPUTSIZE = 256

In [10]:
for i, image_name in enumerate(no_tumour_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'no/' + image_name)
        
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUTSIZE, INPUTSIZE))

        dataset.append(np.array(image))
        label.append(0)

In [11]:
for i, image_name in enumerate(yes_tumour_images):
    if(image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(image_directory + 'yes/' + image_name)
        
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUTSIZE, INPUTSIZE))

        dataset.append(np.array(image))
        label.append(1)

In [12]:
dataset = np.array(dataset)
label = np.array(label)

## Data Splitting

In [13]:
xtrain, xtest, ytrain, ytest = train_test_split(dataset, label, test_size=0.2, random_state=0)

In [14]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(1280, 256, 256, 3)
(320, 256, 256, 3)
(1280,)
(320,)


In [15]:
xtrain = normalize(xtrain, axis=1)
xtest = normalize(xtest, axis=1)

## Model Build

In [16]:
#Basic CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (INPUTSIZE, INPUTSIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(256)) #corresponding to INPUTSIZE
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

#############RESNET50 Model#####################
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# for layer in base_model.layers:
#     layer.trainable = False

# # Add custom top layers for binary classification
# model = Sequential([
#     base_model,
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])



###########RESNET50 Model#######################
# base_model = ResNet50(include_top=False, weights='imagenet')
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation = 'relu')(x)
# predictions = Dense(train_generator.num_classes, activation='sigmoid')(x)
# model = Model(inputs=base_model.input, outputs=predictions)

# for layer in base_model.layers:
#     layer.trainable = False

In [17]:
model.compile(loss= 'binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [18]:
model.fit(xtrain, ytrain, 
          batch_size=16, 
          verbose=1, 
          epochs=3, 
          validation_data=(xtest, ytest), 
          shuffle=False)

Epoch 1/3
80/80 [==============================] - 145s 2s/step - loss: 0.5548 - accuracy: 0.7367 - val_loss: 0.5001 - val_accuracy: 0.7375
Epoch 2/3
80/80 [==============================] - 139s 2s/step - loss: 0.4024 - accuracy: 0.8258 - val_loss: 0.3682 - val_accuracy: 0.8469
Epoch 3/3
80/80 [==============================] - 137s 2s/step - loss: 0.2980 - accuracy: 0.8813 - val_loss: 0.2722 - val_accuracy: 0.8813


In [19]:
model.save('brain_tumour_CNN.h5')

## Model Accuracy

In [20]:
test_loss, test_acc = model.evaluate(xtest, ytest)
print(f"This Model error is: {round((test_loss)*100, 2)}% error")
print(f"This Model scores: {round((test_acc)*100, 2)}% accuracy")

10/10 [==============================] - 8s 727ms/step - loss: 0.2722 - accuracy: 0.8813
This Model error is: 27.22% error
This Model scores: 88.13% accuracy


## Graph Plot

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(xtrain, ytrain, 
          batch_size=16, 
          verbose=1, 
          epochs=3, 
          validation_data=(xtest, ytest), 
          shuffle=False)

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')

fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')

fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()